In [1]:
from paths import get_path_wpp, get_path_collected

import tracemalloc as tmall
tmall.start()

In [2]:
import utils_collection as ut

In [3]:
# PROJECT_NAME="Climate change"
# PROJECT_NAME="COVID-19"

PROJECT_NAME='Media_sample'

In [4]:
# WPP_DATE='2023-04-21'
# WPP_DATE='2023-07-20'
# WPP_DATE='2023-08-16'

# for covid-19
WPP_DATE='2023-09-18'

#----------- for topic-wise
WPP_DATE='2024-01-19' #old
# for topicwise all/sample
WPP_DATE='2024-01-22'

In [5]:
# !dir ../../../../data/shared/jdignazi/wiscom/data

In [6]:
PROCESS_EN=True
PROCESS_OT=True

In [7]:
PATH_INPUT=get_path_wpp(PROJECT_NAME)

PATHNAME_INPUT=f'{PATH_INPUT}project_pages_{WPP_DATE}.csv'
PATHNAME_INPUT=f'{PATH_INPUT}project_pages_{WPP_DATE}_sample10.csv'

In [8]:
PATHNAME_INPUT

'../../../../data/big/jdignazi/Wiscom/collected/Media_sample/project_pages_2024-01-22_sample10.csv'

In [9]:
PATH_OUTPUT=get_path_wpp(PROJECT_NAME)+'{}/' #<--- will be filled with language

In [10]:
PATH_OUTPUT

'../../../../data/big/jdignazi/Wiscom/collected/Media_sample/{}/'

In [11]:
RETURN_ALL_REVISIONS=False

In [12]:
SAMPLE_RUN=False

SUFFIX=''

SAMPLE_PERCENTAGE=1

# SUFFIX+='_'+str(SAMPLE_PERCENTAGE)

In [13]:
if SAMPLE_RUN:
    PATH_OUTPUT+='sample/'

In [14]:
ut.SOURCE_STORAGE=True
ut.SOURCE_STORAGE_COUNT_ONE_PER_LOCAL_CALL=True

In [15]:
ut.RUN_EXPLORE_LANG_TEMPLATES=True
ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT=True

In [16]:
ut.RUN_TEST_ALL=False

ut.RUN_TEST_NOSOURCE_REF_TEXT=True

In [17]:
import pandas as pd

In [18]:
# !pip install urlextract

# Loading Project related pages (all)

In [19]:
WPP_PAGES=pd.read_csv(PATHNAME_INPUT)
WPP_PAGES

,pageid,revision_id,title,ns
0,38877886,1080332169,Las Fierbinți,0
1,62597108,1024208345,Death Come True,0
2,47012099,1062677056,DK (band),0
3,21004385,1052184088,WACR-FM,0
4,8709414,1080177472,Steve Spangler,0
...,...,...,...,...
103219,20208001,1041200033,People of the Cumberland,0
103220,37044406,1077655607,Humberto Elizondo,0
103221,60630116,1022598495,2019 New Zealand Radio Awards,0
103222,10838173,1077441689,Laura Marling,0


### Selecting pages of article (by namespace 0)

In [20]:
WPP_PAGES=WPP_PAGES[WPP_PAGES.ns==0]   

# Retrieve already queried pages

In [21]:
if SAMPLE_RUN:
    print("since this is a sample for testing,") 
    print("the dataframe is replaced silently at every call of this notebook")
    EN_ALREADY_RETRIEVED_PAGES=set()
    OT_ALREADY_RETRIEVED_PAGES=set()
    
else:
    if PROCESS_EN:
        print("looking for english data data in folder:", PATH_OUTPUT.format('english'))
        try:
            EN_ALREADY_RETRIEVED_PAGES=set([row.lang+'$'+row.page \
                                            for _, row in pd.read_csv(f'{PATH_OUTPUT.format("english")}pages_revdata{SUFFIX}.csv').iterrows()])
            print("file found\n")
        except:
            print("file not found\n")
            EN_ALREADY_RETRIEVED_PAGES=set()
            
    if PROCESS_OT:
        print("looking for other languages data data in folder:", PATH_OUTPUT.format('other_languages'))
        try:    
            OT_ALREADY_RETRIEVED_PAGES=set([row.lang+'$'+row.page \
                                            for _, row in pd.read_csv(f'{PATH_OUTPUT.format("other_languages")}pages_revdata{SUFFIX}.csv').iterrows()])    
            print("file found\n")
        except:
            print("file not found\n")
            OT_ALREADY_RETRIEVED_PAGES=set()

looking for english data data in folder: ../../../../data/big/jdignazi/Wiscom/collected/Media_sample/english/
file found

looking for other languages data data in folder: ../../../../data/big/jdignazi/Wiscom/collected/Media_sample/other_languages/
file found



In [22]:
print("english retrieved pages:", len(EN_ALREADY_RETRIEVED_PAGES) )
print("other languages retrieved pages:", len(OT_ALREADY_RETRIEVED_PAGES) )

english retrieved pages: 18680
other languages retrieved pages: 70323


#### Select pages for the sample

In [23]:
if SAMPLE_RUN:
    SAMPLE_PAGES=WPP_PAGES.title.values[list( range(0, len(WPP_PAGES), int(100/SAMPLE_PERCENTAGE)) )]
else:
    SAMPLE_PAGES=WPP_PAGES.title.values

# Extract url revisions from project pages + lang links

In [24]:
print("data will be saved in", PATH_OUTPUT.format("LANGUAGE_SPECIFIER"))
print("suffix:", SUFFIX)

data will be saved in ../../../../data/big/jdignazi/Wiscom/collected/Media_sample/LANGUAGE_SPECIFIER/
suffix: 


In [25]:
from time import time

In [26]:
# TO DO
# add page to already retrieved once has finished
# timer whole process inside iterator
# cleaner checkpoint saving for explore lang informations

In [27]:
import pickle

# for exploratory purposes
# if PROCESS_EN and PROCESS_OT: _t_prefix='all'
# if not PROCESS_EN and PROCESS_OT: _t_prefix='ot'
# if PROCESS_EN and not PROCESS_OT: _t_prefix='en'

In [ ]:
%%time

EN_ADDED_COUNTER=0
OT_ADDED_COUNTER=0

for n_page, en_page_title in enumerate(SAMPLE_PAGES):
    main_time=time()
    if PROCESS_EN:
        if not f'en${en_page_title}' in EN_ALREADY_RETRIEVED_PAGES:
            en_t_start=time()
            ALL_DATA =ut.get_sources_revisions(en_page_title, 'en', return_all_revisions=RETURN_ALL_REVISIONS,
                                               verbose=True, 
                                               verbose_prefix=f"------------{n_page+1:>4d} of {len(SAMPLE_PAGES)} ")

            mode, header='a', False
            if n_page==0 and not len(EN_ALREADY_RETRIEVED_PAGES):
                mode, header='w', True
            ut.save_rv_data(ALL_DATA, PATH_OUTPUT.format('english'), SUFFIX, mode, header)
            EN_ADDED_COUNTER+=1    
            ut._update_timer('___pipeline_english', en_t_start)
            EN_ALREADY_RETRIEVED_PAGES.update([f'en${en_page_title}'])
            print()

    if PROCESS_OT:
        ot_t_start=time()
        LL_DATA=ut.get_language_links_data(en_page_title)
        for n_ll, ll in enumerate(LL_DATA):
            LANG=ll['lang']
            PAGE=ll['*']
            
            if f'{LANG}${PAGE}' in OT_ALREADY_RETRIEVED_PAGES: continue

            ALL_DATA =ut.get_sources_revisions(PAGE, LANG, return_all_revisions=RETURN_ALL_REVISIONS,
                                               verbose=True, 
                                               verbose_prefix=f"{n_ll+1:>3} of {len(LL_DATA):<3} ({n_page+1:>4d} of {len(SAMPLE_PAGES)})")

            mode, header='a', False
            if n_page==0 and n_ll==0 and not len(OT_ALREADY_RETRIEVED_PAGES):
                mode, header='w', True
            ut.save_rv_data(ALL_DATA, PATH_OUTPUT.format('other_languages'), SUFFIX, mode, header)
            OT_ADDED_COUNTER+=1    
            print('\n next ll...', end='\r')
            OT_ALREADY_RETRIEVED_PAGES.update([f'{LANG}${PAGE}'])

        ut._update_timer('___pipeline_other_languages', ot_t_start)

#         for exploratory purposes
# #     if n_page%10==0:
#     if ut.RUN_EXPLORE_LANG_TEMPLATES:
#         with open(f'../data/explore_lang/{_t_prefix}_lang_template_data{SUFFIX}_new.pickle', 'wb') as handle:
#             pickle.dump(ut.EXPLORE_LANG_TEMPLATES, handle)

#     if ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT:    
#         with open(f'../data/explore_lang/{_t_prefix}_lang_nosource_ref_text{SUFFIX}_new.pickle', 'wb') as handle:
#             pickle.dump(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT, handle)
        
    ut._update_timer('whole_process', main_time)

langlinks not found for request of page WACR-FM
full response data:

{'batchcomplete': '', 'query': {'pages': {'21004385': {'pageid': 21004385, 'ns': 0, 'title': 'WACR-FM'}}}}

langlinks not found for request of page WikipediaVision
full response data:

{'batchcomplete': '', 'query': {'pages': {'22127174': {'pageid': 22127174, 'ns': 0, 'title': 'WikipediaVision'}}}}

langlinks not found for request of page The Promised Land (Muzz album)
full response data:

{'batchcomplete': '', 'query': {'pages': {'64665830': {'pageid': 64665830, 'ns': 0, 'title': 'The Promised Land (Muzz album)'}}}}

langlinks not found for request of page Colouz Uv Sound
full response data:

{'batchcomplete': '', 'query': {'pages': {'28088876': {'pageid': 28088876, 'ns': 0, 'title': 'Colouz Uv Sound'}}}}

langlinks not found for request of page The Law of Freedom in a Platform
full response data:

{'batchcomplete': '', 'query': {'pages': {'27659535': {'pageid': 27659535, 'ns': 0, 'title': 'The Law of Freedom in a Pl

langlinks not found for request of page Soy (Víctor Manuelle album)
full response data:

{'batchcomplete': '', 'query': {'pages': {'20007885': {'pageid': 20007885, 'ns': 0, 'title': 'Soy (Víctor Manuelle album)'}}}}

langlinks not found for request of page The Paradise Novels
full response data:

{'batchcomplete': '', 'query': {'pages': {'30157934': {'pageid': 30157934, 'ns': 0, 'title': 'The Paradise Novels'}}}}

langlinks not found for request of page Principles of Nature
full response data:

{'batchcomplete': '', 'query': {'pages': {'25415613': {'pageid': 25415613, 'ns': 0, 'title': 'Principles of Nature'}}}}

langlinks not found for request of page List of songs recorded by DJ Quik
full response data:

{'batchcomplete': '', 'query': {'pages': {'31397190': {'pageid': 31397190, 'ns': 0, 'title': 'List of songs recorded by DJ Quik'}}}}

langlinks not found for request of page Driven (2008)
full response data:

{'batchcomplete': '', 'query': {'pages': {'19581559': {'pageid': 19581559, 

langlinks not found for request of page Lucas Leyva
full response data:

{'batchcomplete': '', 'query': {'pages': {'70193890': {'pageid': 70193890, 'ns': 0, 'title': 'Lucas Leyva'}}}}

langlinks not found for request of page The Secret Life of Nora
full response data:

{'batchcomplete': '', 'query': {'pages': {'31328531': {'pageid': 31328531, 'ns': 0, 'title': 'The Secret Life of Nora'}}}}

langlinks not found for request of page Dumfries & Galloway Standard
full response data:

{'batchcomplete': '', 'query': {'pages': {'20924115': {'pageid': 20924115, 'ns': 0, 'title': 'Dumfries & Galloway Standard'}}}}

langlinks not found for request of page I Plan to Stay a Believer
full response data:

{'batchcomplete': '', 'query': {'pages': {'42875216': {'pageid': 42875216, 'ns': 0, 'title': 'I Plan to Stay a Believer'}}}}

langlinks not found for request of page Johannesburg (EP)
full response data:

{'batchcomplete': '', 'query': {'pages': {'50885839': {'pageid': 50885839, 'ns': 0, 'title': 'J

langlinks not found for request of page Mirage (M. Pokora song)
full response data:

{'batchcomplete': '', 'query': {'pages': {'34996961': {'pageid': 34996961, 'ns': 0, 'title': 'Mirage (M. Pokora song)'}}}}

langlinks not found for request of page Chill Pill
full response data:

{'batchcomplete': '', 'query': {'pages': {'16410939': {'pageid': 16410939, 'ns': 0, 'title': 'Chill Pill'}}}}

langlinks not found for request of page Theatre Night
full response data:

{'batchcomplete': '', 'query': {'pages': {'42694226': {'pageid': 42694226, 'ns': 0, 'title': 'Theatre Night'}}}}

langlinks not found for request of page Big Deal (game show)
full response data:

{'batchcomplete': '', 'query': {'pages': {'4538216': {'pageid': 4538216, 'ns': 0, 'title': 'Big Deal (game show)'}}}}

langlinks not found for request of page We Are Hear
full response data:

{'batchcomplete': '', 'query': {'pages': {'62355566': {'pageid': 62355566, 'ns': 0, 'title': 'We Are Hear'}}}}

langlinks not found for request o

langlinks not found for request of page Liber miraculorum sancte Fidis
full response data:

{'batchcomplete': '', 'query': {'pages': {'55961914': {'pageid': 55961914, 'ns': 0, 'title': 'Liber miraculorum sancte Fidis'}}}}

langlinks not found for request of page TV Showboat
full response data:

{'batchcomplete': '', 'query': {'pages': {'42757028': {'pageid': 42757028, 'ns': 0, 'title': 'TV Showboat'}}}}

langlinks not found for request of page Listen to Me (album)
full response data:

{'batchcomplete': '', 'query': {'pages': {'26070952': {'pageid': 26070952, 'ns': 0, 'title': 'Listen to Me (album)'}}}}

langlinks not found for request of page 5IT
full response data:

{'batchcomplete': '', 'query': {'pages': {'13605770': {'pageid': 13605770, 'ns': 0, 'title': '5IT'}}}}

langlinks not found for request of page Bhaage Re Mann
full response data:

{'batchcomplete': '', 'query': {'pages': {'48431987': {'pageid': 48431987, 'ns': 0, 'title': 'Bhaage Re Mann'}}}}

langlinks not found for reque

...

langlinks not found for request of page Spencer Hughes (audio engineer)
full response data:

{'batchcomplete': '', 'query': {'pages': {'63410704': {'pageid': 63410704, 'ns': 0, 'title': 'Spencer Hughes (audio engineer)'}}}}

langlinks not found for request of page Off Center (short story collection)
full response data:

{'batchcomplete': '', 'query': {'pages': {'24508144': {'pageid': 24508144, 'ns': 0, 'title': 'Off Center (short story collection)'}}}}

langlinks not found for request of page Love Made Me Stupid
full response data:

{'batchcomplete': '', 'query': {'pages': {'27173807': {'pageid': 27173807, 'ns': 0, 'title': 'Love Made Me Stupid'}}}}

langlinks not found for request of page XYZ Entertainments
full response data:

{'batchcomplete': '', 'query': {'pages': {'61282283': {'pageid': 61282283, 'ns': 0, 'title': 'XYZ Entertainments'}}}}

langlinks not found for request of page KITG
full response data:

{'batchcomplete': '', 'query': {'pages': {'33806015': {'pageid': 33806015, '

langlinks not found for request of page The Deadly Syndrome
full response data:

{'batchcomplete': '', 'query': {'pages': {'12979877': {'pageid': 12979877, 'ns': 0, 'title': 'The Deadly Syndrome'}}}}

langlinks not found for request of page Characters of Ackley Bridge
full response data:

{'batchcomplete': '', 'query': {'pages': {'67191472': {'pageid': 67191472, 'ns': 0, 'title': 'Characters of Ackley Bridge'}}}}

langlinks not found for request of page Truth (British periodical)
full response data:

{'batchcomplete': '', 'query': {'pages': {'31790082': {'pageid': 31790082, 'ns': 0, 'title': 'Truth (British periodical)'}}}}

langlinks not found for request of page KAML-FM
full response data:

{'batchcomplete': '', 'query': {'pages': {'14991550': {'pageid': 14991550, 'ns': 0, 'title': 'KAML-FM'}}}}

langlinks not found for request of page Cybertechnology
full response data:

{'batchcomplete': '', 'query': {'pages': {'61019829': {'pageid': 61019829, 'ns': 0, 'title': 'Cybertechnology'}}}

------------18691 of 103224 ; page Telecommunications in Nigeria (en); scanned 181 revisions of 181/277; collected 249 sources revisions...
  1 of 5   (18691 of 103224); page Telekommunikation in Nigeria (de); scanned 9 revisions of 9/20; collected 25 sources revisions...
  2 of 5   (18691 of 103224); page Télécommunications au Nigeria (fr); scanned 8 revisions of 8/17; collected 33 sources revisions...
  3 of 5   (18691 of 103224); page Հեռահաղորդակցությունը Նիգերիայում (hy); scanned 10 revisions of 10/10; collected 3 sources revisions...
  4 of 5   (18691 of 103224); page Nigerijos ryšiai (lt); scanned 7 revisions of 7/9; collected 2 sources revisions...
  5 of 5   (18691 of 103224); page Связь в Нигерии (ru); scanned 37 revisions of 37/38; collected 11 sources revisions...
------------18692 of 103224 ; page May the Force Be with You Always (en); scanned 12 revisions of 12/14; collected 1 sources revisions...
langlinks not found for request of page May the Force Be with You Always
fu

  2 of 6   (18714 of 103224); page Arizona (film fra 1913) (da); scanned 2 revisions of 2/4; collected 2 sources revisions...
  3 of 6   (18714 of 103224); page آریزونا (فیلم ۱۹۱۳) (fa); scanned 12 revisions of 12/20; collected 0 sources revisions...
  4 of 6   (18714 of 103224); page Arizona (film, 1913) (fr); scanned 22 revisions of 22/23; collected 3 sources revisions...
  5 of 6   (18714 of 103224); page Arizona (fim, 1913) (ht); scanned 1 revisions of 1/1; collected 3 sources revisions...
  6 of 6   (18714 of 103224); page Arizona (1913) (nb); scanned 3 revisions of 3/6; collected 2 sources revisions...
------------18715 of 103224 ; page List of awards and nominations received by The Flash (en); scanned 50 revisions of 50/317; collected 256 sources revisions...
  1 of 2   (18715 of 103224); page فهرست جایزه‌ها و نامزدی‌های فلش (fa); scanned 8 revisions of 8/15; collected 79 sources revisions...
  2 of 2   (18715 of 103224); page Lista de prêmios e indicações recebidos por The Flas

 30 of 35  (18745 of 103224); page Découvertes Gallimard (sv); scanned 7 revisions of 7/24; collected 2 sources revisions...
 31 of 35  (18745 of 103224); page Découvertes Gallimard (tr); scanned 21 revisions of 21/30; collected 9 sources revisions...
 32 of 35  (18745 of 103224); page 發現之旅 (wuu); scanned 2 revisions of 2/44; collected 49 sources revisions...
 33 of 35  (18745 of 103224); page 發現之旅叢書 (zh); scanned 32 revisions of 32/463; collected 258 sources revisions...
 34 of 35  (18745 of 103224); page 發現之旅 (zh-classical); scanned 8 revisions of 8/35; collected 13 sources revisions...
 35 of 35  (18745 of 103224); page 發現之旅 (zh-yue); scanned 30 revisions of 30/220; collected 61 sources revisions...
------------18746 of 103224 ; page Han Ningen (en); scanned 7 revisions of 7/14; collected 10 sources revisions...
langlinks not found for request of page Han Ningen
full response data:

{'batchcomplete': '', 'query': {'pages': {'69610406': {'pageid': 69610406, 'ns': 0, 'title': 'Han Nin

In [ ]:
ut.N_RESPONSE_EXCEPTIONS

In [ ]:
ut.N_RESPONSE_EXCEPTIONS_LL

In [ ]:
OT_ADDED_COUNTER

In [ ]:
EN_ADDED_COUNTER

In [ ]:
ut.GLOBAL_COUNT_RETAINED

In [ ]:
ut.GLOBAL_COUNT_RETAINED/(ut.GLOBAL_COUNT_SKIPPED+ut.GLOBAL_COUNT_RETAINED)

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
ut.dir_performance(normalize='whole_process') 

### Saving exploration results (if required)

In [ ]:
# import pickle

# if PROCESS_EN and PROCESS_OT: _t_prefix='all'
# if not PROCESS_EN and PROCESS_OT: _t_prefix='ot'
# if PROCESS_EN and not PROCESS_OT: _t_prefix='en'
    
# if ut.RUN_EXPLORE_LANG_TEMPLATES:
#     with open(f'../data/explore_lang/{_t_prefix}_lang_template_data{SUFFIX}_part_3.pickle', 'wb') as handle:
#         pickle.dump(ut.EXPLORE_LANG_TEMPLATES, handle)
        
# if ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT:    
#     with open(f'../data/explore_lang/{_t_prefix}_lang_nosource_ref_text{SUFFIX}_part_3.pickle', 'wb') as handle:
#         pickle.dump(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT, handle)

In [ ]:
# # Load data (deserialize)
# with open(f'../data/other_languages_template_data{SUFFIX}.pickle', 'rb') as handle:
#     this = pickle.load(handle)

#### Extimate parallelization limitations
- Tracing request rate (can use utils timer and counter)
- Tracing memory usage (can use tmall)

In [ ]:
print("request per hour (revisions):", \
      ut.TEMP_COUNTERS['_request_revision_data']/(ut.TEMP_TIMERS['whole_process']/60**2)," requests per hour")
print("request per hour (revisions+language links):", \
      (ut.TEMP_COUNTERS['_request_revision_data']+ut.TEMP_COUNTERS['request_language_links'])/(ut.TEMP_TIMERS['whole_process']/60**2)," requests per hour")

In [ ]:
_, PT_MPK=tmall.get_traced_memory() 

print(f" memory peak: {PT_MPK/(10**6)} Mb")

In [ ]:
_/10**6

In [ ]:
import sys
sys.getsizeof(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT)

# Extimate by sample proportion (rough)

In [ ]:
if SAMPLE_RUN:
    print(f"extimated total time: {ut.TEMP_TIMERS['whole_process']*100/SAMPLE_PERCENTAGE/60**2:.2f} hrs")

# Code performances

In [ ]:
ut.dir_performance(normalize='whole_process')

In [ ]:
print("total pages analysed in this sample (english):", len(SAMPLE_PAGES))
print(f"{len(SAMPLE_PAGES)/len(WPP_PAGES)*100:.2f}% of the total")

In [ ]:
print("total pages analysed in this sample (en+ot):", ut.TEMP_COUNTERS['pipeline_sources'])
# print(f"{ut.TEMP_COUNTERS['_sources_extraction']/EXTIMATED_TOT_REVISIONS*100:.2f}% of the total")

In [ ]:
print("total revisions analysed in this sample (en+ot):", ut.TEMP_COUNTERS['_sources_extraction'])

In [ ]:
print("total time of this sample process:", ut.TEMP_TIMERS['whole_process']/60**2, 'hrs')

In [ ]:
print("average time per page (english):", 
      ut.TEMP_TIMERS['___pipeline_english']/(ut.TEMP_COUNTERS['___pipeline_english'])/60," min per page")

In [ ]:
print("average time per page (other_languages):", 
      ut.TEMP_TIMERS['___pipeline_other_languages']/(ut.TEMP_COUNTERS['___pipeline_other_languages'])/60," min per page")

In [ ]:
print("average time per revision (english+other_languages):",\
      ut.TEMP_TIMERS['whole_process']/(ut.TEMP_COUNTERS['_sources_extraction'])," seconds per revision (including request time)")

In [ ]:
print("average time TO PROCESS one revision:", ut.TEMP_TIMERS['_sources_extraction']/(ut.TEMP_COUNTERS['_sources_extraction'])," seconds per revision")

# Extimate by revision (if tot revision in known)

#### Extimate total time
- expected_tot revisions * average revision time =expected_tot_revisions * (sample_n_revisions/sample_process_time)
- - sample_n_revisions is given by ut.TEMP_COUNTERS(\'_sources_extraction\') - tracing get_sources2locations calls
- - sample_process_time is ut.TEMP_TIMERS('whole_process') (accounting for request time as well)
- - expected_tot_revisions: counting them in wpp_to_n_revisions.ipynb

In [ ]:
# unknown for other languages
# EXTIMATED_TOT_REVISIONS=1483462 #manually added from results of wpp_to_n_revisions

<!-- Can be an overextimation, since the time to process one revision is dependent on page length and number of templates/ref statements to be found -->

In [ ]:
# print("extimated total time:", 
#       EXTIMATED_TOT_REVISIONS*ut.TEMP_TIMERS['whole_process']/(ut.TEMP_COUNTERS['_sources_extraction'])/60**2, 'hrs')

<!-- With the sample of the first 40 page, I expect less time since the first one have more edits and more sources inside -->

Info to retain

- **SECTION WHERE THE SOURCE IS USED?** Worthwile I guess
- **WHAT TO DO WITH PAGES THAT ARE JUST REDIRECT?** Not worthwile I guess
- flag some revision? can be done later 
- bot activity? can be done later

Not clear to me (wiki api)

- I do get 50 revision per request, while rvlim='max' is supposed to provide 500 (works with 500 for the wpp_to_n_revisions indeed)
- Is there a request limit for wikipedia action API?
Yes? https://api.wikimedia.org/wiki/Documentation/Getting_started/Rate_limits#Anonymous_requests 
- - Am I anonymus?
- - is this rate limit applicable to this request?

Better pipeline
- Parallelization 
- Asyncronous process to save request time (around 20% time decrease)
- sources2locs algorithm
- - retrieve page differences before comparing versions?
- - ignore somewhat bad revision? [generally no because that is a signal from a user] Maybe revisions flagged as "minor"? If we are sure that a source is not modified in minor revisions

structured data:

- urlsrv dataframe
- - identifier of page (is it implicit in revision_id?)
- - identifier of user (is it implicit in revision_id? Is it worth to de-structure it away?)
- - identifier of revision
- - identifier of url
- - added/removed 

- Page data:
- - identifier of page
- - page id, page name
- - whatever

- User data:
- - identifier of user
- - user_id, username
- - whatever

- Revision data
- - identifier of revision
- - parent_id
- - timestamp
- - user?
- - page?

- URLS data:
- - identifier of url
- - url status
- - domain
- - destination_url
- - destination_domain 

